In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from google.colab import drive

drive.mount('/content/gdrive/')
demographic_data = pd.read_csv('/content/gdrive/My Drive/KPMG/CustomerDemographic.csv', index_col='customer_id')
address_data = pd.read_csv('/content/gdrive/My Drive/KPMG/Demographic.csv', index_col='customer_id')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wE5G-JG0FaZB3_jf3kZrHOft06QqzQOlw64Gx9r7Sv0m8tDnEa-f0o
Mounted at /content/gdrive/


In [ ]:
demographic = demographic_data[['gender',	'DOB',	'job_industry_category',	'owns_car', 'wealth_segment',	'past_3_years_bike_related_purchases']]
demographic['DOB'] = pd.to_datetime(demographic.DOB)

demographic['AGE'] = ((pd.datetime.datetime.now()- demographic.DOB)).dt.days

demographic['AGE'] = np.floor(demographic.AGE / 365)
demographic = demographic[demographic.AGE <= 100]

demographic.sort_values('customer_id', inplace = True)
demographic.dropna(how = 'any', inplace = True)

demographic.gender.replace('F', 'Female', inplace = True)
demographic.gender.replace('Femal', 'Female', inplace = True)
demographic.gender.replace('M', 'Male', inplace = True)

demographic = demographic[['AGE', 'gender',	'job_industry_category',	'owns_car', 'wealth_segment',	'past_3_years_bike_related_purchases']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [ ]:
state = pd.DataFrame(address_data.state)

state.state.replace('New South Wales', 'NSW', inplace = True)
state.state.replace('Victoria', 'VIC', inplace = True)

In [ ]:
data = pd.merge(demographic, state, left_on='customer_id', right_on='customer_id')

age_bucket = [0, 20, 40, 60, 80, 100]
age_groups = ['Age : 0 - 20', 'Age : 21 - 40', 'Age : 41 - 60', 'Age : 61 - 80', 'Age : 81 - 100']
data['Age_Groups'] = pd.cut(data['AGE'], age_bucket, labels=age_groups)

data = data[['Age_Groups', 'gender', 'job_industry_category', 'state',  'owns_car', 'wealth_segment', 'past_3_years_bike_related_purchases']]

data.isna().sum()

Age_Groups                             0
gender                                 0
job_industry_category                  0
state                                  0
owns_car                               0
wealth_segment                         0
past_3_years_bike_related_purchases    0
dtype: int64

In [ ]:
print (data.Age_Groups.value_counts(sort = False, normalize = True))

Age : 0 - 20      0.026437
Age : 21 - 40     0.370427
Age : 41 - 60     0.506609
Age : 61 - 80     0.095911
Age : 81 - 100    0.000615
Name: Age_Groups, dtype: float64


In [ ]:
print (data.gender.value_counts(sort = False, normalize = True))

Female    0.518906
Male      0.481094
Name: gender, dtype: float64


In [ ]:
print (data.job_industry_category.value_counts(sort = False, normalize = True))

IT                    0.046111
Argiculture           0.034737
Manufacturing         0.244697
Property              0.081771
Entertainment         0.041808
Retail                0.110052
Health                0.183215
Telecommunications    0.022133
Financial Services    0.235475
Name: job_industry_category, dtype: float64


In [ ]:
print (data.state.value_counts(sort = False, normalize = True))

VIC    0.256994
QLD    0.207808
NSW    0.535198
Name: state, dtype: float64


In [ ]:
print (data.owns_car.value_counts(sort = False, normalize = True))

Yes    0.509376
No     0.490624
Name: owns_car, dtype: float64


In [ ]:
print (data.wealth_segment.value_counts(sort = False, normalize = True))

Affluent Customer    0.244082
High Net Worth       0.253919
Mass Customer        0.501998
Name: wealth_segment, dtype: float64


In [ ]:
data_scores = data[['Age_Groups', 'gender', 'job_industry_category', 'state',  'owns_car', 'wealth_segment']]

data_scores.Age_Groups.replace('Age : 0 - 20', 0.026437, inplace = True)
data_scores.Age_Groups.replace('Age : 21 - 40', 0.371350, inplace = True)
data_scores.Age_Groups.replace('Age : 41 - 60', 0.505994, inplace = True)
data_scores.Age_Groups.replace('Age : 61 - 80', 0.095604, inplace = True)
data_scores.Age_Groups.replace('Age : 81 - 100', 0.000615, inplace = True)

data_scores.gender.replace('Male', 0.481094, inplace = True)
data_scores.gender.replace('Female', 0.518906, inplace = True)

data_scores.job_industry_category.replace('Entertainment', 0.041808, inplace = True)
data_scores.job_industry_category.replace('Manufacturing', 0.244697, inplace = True)
data_scores.job_industry_category.replace('Health', 0.183215, inplace = True)
data_scores.job_industry_category.replace('IT', 0.046111, inplace = True)
data_scores.job_industry_category.replace('Property', 0.081771, inplace = True)
data_scores.job_industry_category.replace('Financial Services', 0.235475, inplace = True)
data_scores.job_industry_category.replace('Argiculture', 0.034737, inplace = True)
data_scores.job_industry_category.replace('Retail', 0.110052, inplace = True)
data_scores.job_industry_category.replace('Telecommunications', 0.022133, inplace = True)

data_scores.state.replace('QLD', 0.207808, inplace = True)
data_scores.state.replace('VIC', 0.256994, inplace = True)
data_scores.state.replace('NSW', 0.535198, inplace = True)

data_scores.owns_car.replace('Yes', 0.509376, inplace = True)
data_scores.owns_car.replace('No', 0.490624, inplace = True)

data_scores.wealth_segment.replace('High Net Worth', 0.253919, inplace = True)
data_scores.wealth_segment.replace('Affluent Customer', 0.244082, inplace = True)
data_scores.wealth_segment.replace('Mass Customer', 0.501998, inplace = True)

data_scores.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Age_Groups,gender,job_industry_category,state,owns_car,wealth_segment
customer_id,,,,,,
1,0.095604,0.518906,0.183215,0.535198,0.509376,0.501998
2,0.371350,0.481094,0.235475,0.535198,0.509376,0.501998
4,0.505994,0.481094,0.046111,0.207808,0.490624,0.501998
6,0.505994,0.481094,0.110052,0.256994,0.509376,0.253919
7,0.505994,0.518906,0.235475,0.535198,0.509376,0.244082


In [ ]:
data_scores['Age_Groups'] = data_scores.Age_Groups.astype(float)

data_scores['Total_Score'] = data_scores.Age_Groups * data_scores.gender * data_scores.job_industry_category * data_scores.state *	data_scores.owns_car * data_scores.wealth_segment	

data_scores.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Age_Groups,gender,job_industry_category,state,owns_car,wealth_segment,Total_Score
customer_id,,,,,,,
1,0.095604,0.518906,0.183215,0.535198,0.509376,0.501998,0.001244
2,0.371350,0.481094,0.235475,0.535198,0.509376,0.501998,0.005757
4,0.505994,0.481094,0.046111,0.207808,0.490624,0.501998,0.000575
6,0.505994,0.481094,0.110052,0.256994,0.509376,0.253919,0.000890
7,0.505994,0.518906,0.235475,0.535198,0.509376,0.244082,0.004114


In [ ]:
print (data_scores.Total_Score.describe())
print (data_scores.Total_Score.median())

count    3253.000000
mean        0.002610
std         0.002176
min         0.000002
25%         0.000889
50%         0.001979
75%         0.003767
max         0.008793
Name: Total_Score, dtype: float64
0.001979455929162918


In [ ]:
data_scores_finalized = data[['Age_Groups', 'gender', 'job_industry_category', 'state',  'owns_car', 'wealth_segment']]
data_scores_finalized['Total_Score'] = data_scores['Total_Score']

data_scores_finalized.head()

,Age_Groups,gender,job_industry_category,state,owns_car,wealth_segment,Total_Score
customer_id,,,,,,,
1,Age : 61 - 80,Female,Health,NSW,Yes,Mass Customer,0.001244
2,Age : 21 - 40,Male,Financial Services,NSW,Yes,Mass Customer,0.005757
4,Age : 41 - 60,Male,IT,QLD,No,Mass Customer,0.000575
6,Age : 41 - 60,Male,Retail,VIC,Yes,High Net Worth,0.000890
7,Age : 41 - 60,Female,Financial Services,NSW,Yes,Affluent Customer,0.004114


In [ ]:
for ind, row in data_scores_finalized.iterrows():
  if (data_scores_finalized.loc[ind, 'Total_Score'] >= 0.003767) :
    data_scores_finalized.loc[ind, 'Customer_Class'] = 'Very Important'
  if ((data_scores_finalized.loc[ind, 'Total_Score'] < 0.003767) & (data_scores_finalized.loc[ind, 'Total_Score'] >= 0.001979)) :
    data_scores_finalized.loc[ind, 'Customer_Class'] = 'Important'
  if ((data_scores_finalized.loc[ind, 'Total_Score'] < 0.001979) & (data_scores_finalized.loc[ind, 'Total_Score'] >= 0.000889)) :
    data_scores_finalized.loc[ind, 'Customer_Class'] = 'General'
  if (data_scores_finalized.loc[ind, 'Total_Score'] < 0.000889) :
    data_scores_finalized.loc[ind, 'Customer_Class'] = 'Not Important'

training_dataset = data_scores_finalized[['Age_Groups', 'gender', 'job_industry_category', 'state',  'owns_car', 'wealth_segment', 'Customer_Class']] 
training_dataset.head(10)

,Age_Groups,gender,job_industry_category,state,owns_car,wealth_segment,Customer_Class
customer_id,,,,,,,
1,Age : 61 - 80,Female,Health,NSW,Yes,Mass Customer,General
2,Age : 21 - 40,Male,Financial Services,NSW,Yes,Mass Customer,Very Important
4,Age : 41 - 60,Male,IT,QLD,No,Mass Customer,Not Important
6,Age : 41 - 60,Male,Retail,VIC,Yes,High Net Worth,General
7,Age : 41 - 60,Female,Financial Services,NSW,Yes,Affluent Customer,Very Important
9,Age : 41 - 60,Female,Argiculture,NSW,Yes,Affluent Customer,Not Important
11,Age : 61 - 80,Male,Property,VIC,No,Mass Customer,Not Important
12,Age : 21 - 40,Male,Manufacturing,QLD,No,Mass Customer,Important
13,Age : 61 - 80,Male,Financial Services,VIC,Yes,High Net Worth,Not Important


In [ ]:
training_dataset.to_csv('/content/gdrive/My Drive/KPMG/Training_Dataset.csv')

In [ ]:
data_scores_finalized.Customer_Class.value_counts(ascending = True)

General           806
Very Important    811
Not Important     813
Important         823
Name: Customer_Class, dtype: int64

In [ ]:
data.groupby(['Age_Groups']).past_3_years_bike_related_purchases.sum()

Age_Groups
Age : 0 - 20       4111
Age : 21 - 40     61163
Age : 41 - 60     79302
Age : 61 - 80     14788
Age : 81 - 100      123
Name: past_3_years_bike_related_purchases, dtype: int64

In [ ]:
data.groupby(['gender']).past_3_years_bike_related_purchases.sum()

gender
Female    81008
Male      78479
Name: past_3_years_bike_related_purchases, dtype: int64

In [ ]:
data.groupby(['job_industry_category']).past_3_years_bike_related_purchases.sum()

job_industry_category
Argiculture            5945
Entertainment          6320
Financial Services    37115
Health                29856
IT                     7352
Manufacturing         39121
Property              13233
Retail                17301
Telecommunications     3244
Name: past_3_years_bike_related_purchases, dtype: int64

In [ ]:
data.groupby(['state']).past_3_years_bike_related_purchases.sum()

state
NSW    85962
QLD    33405
VIC    40120
Name: past_3_years_bike_related_purchases, dtype: int64

In [ ]:
data.groupby(['owns_car']).past_3_years_bike_related_purchases.sum()

owns_car
No     78242
Yes    81245
Name: past_3_years_bike_related_purchases, dtype: int64

In [ ]:
data.groupby(['wealth_segment']).past_3_years_bike_related_purchases.sum() 

wealth_segment
Affluent Customer    38976
High Net Worth       39655
Mass Customer        80856
Name: past_3_years_bike_related_purchases, dtype: int64